In [ ]:
# ===== System-managed metadata (auto-generated, understand to edit) =====
# @node_type: data_source
# @node_id: load_orders_data
# @execution_status: pending_validation
# @name: Load Orders Data
# ===== End of system-managed metadata =====


In [ ]:
# ===== System-managed metadata (auto-generated, understand to edit) =====
# @node_type: data_source
# @node_id: load_customers_data
# @execution_status: validated
# @name: Load Customers Data
# ===== End of system-managed metadata =====import pandas as pd
import os

# Load customers data from CSV
customers_path = os.path.join(os.getcwd(), 'customers.csv')
load_customers_data = pd.read_csv(customers_path)
load_customers_data['register_date'] = pd.to_datetime(load_customers_data['register_date'])


In [ ]:
# ===== System-managed metadata (auto-generated, understand to edit) =====
# @node_type: data_source
# @node_id: load_products_data
# @execution_status: not_executed
# @name: Load Products Data
# ===== End of system-managed metadata =====
import pandas as pd
import os

# Load products data from CSV
products_path = os.path.join(os.getcwd(), 'products.csv')
load_products_data = pd.read_csv(products_path)



In [ ]:
# ===== System-managed metadata (auto-generated, understand to edit) =====
# @node_type: data_source
# @node_id: load_promotions_data
# @execution_status: validated
# @name: Load Promotions Data
# ===== End of system-managed metadata =====
import pandas as pd
import os

# Load promotions data from CSV
promotions_path = os.path.join(os.getcwd(), 'promotions.csv')
load_promotions_data = pd.read_csv(promotions_path)
load_promotions_data['start_date'] = pd.to_datetime(load_promotions_data['start_date'])
load_promotions_data['end_date'] = pd.to_datetime(load_promotions_data['end_date'])



In [ ]:
# ===== System-managed metadata (auto-generated, understand to edit) =====
# @node_type: tool
# @node_id: tool_time_aggregation
# @execution_status: not_executed
# @name: Tool: Time Aggregation Functions
# ===== End of system-managed metadata =====
import pandas as pd

def tool_time_aggregation(df, date_col, value_col, freq='D'):
    """Aggregate data by time period (Day, Week, Month)"""
    df_temp = df.copy()
    df_temp[date_col] = pd.to_datetime(df_temp[date_col])
    result = df_temp.groupby(pd.Grouper(key=date_col, freq=freq))[value_col].sum()
    return result.reset_index()

# Tool is now available for use in other compute nodes


In [ ]:
# ===== System-managed metadata (auto-generated, understand to edit) =====
# @node_type: tool
# @node_id: tool_cohort_analysis
# @execution_status: not_executed
# @name: Tool: Cohort Analysis
# ===== End of system-managed metadata =====
import pandas as pd

def tool_cohort_analysis(transactions_df, customer_df, cohort_dimension):
    """Group customers by cohort dimension for comparison"""
    merged = transactions_df.merge(customer_df, on='customer_id')
    cohort_stats = merged.groupby(cohort_dimension).agg({
        'customer_id': 'nunique',
        'total_amount': ['sum', 'mean', 'count'],
        'quantity': 'sum'
    }).round(2)
    return cohort_stats

# Tool is now available for use in other compute nodes


In [ ]:
# ===== System-managed metadata (auto-generated, understand to edit) =====
# @node_type: tool
# @node_id: tool_category_metrics
# @execution_status: not_executed
# @name: Tool: Category Metrics
# ===== End of system-managed metadata =====
import pandas as pd

def tool_category_metrics(orders_df, products_df):
    """Calculate category-level performance metrics"""
    merged = orders_df.merge(products_df[['product_id', 'cost']], on='product_id')
    metrics = merged.groupby('category').agg({
        'total_amount': 'sum',
        'quantity': 'sum',
        'order_id': 'count',
        'cost': lambda x: x.sum()
    }).rename(columns={'order_id': 'order_count'})
    metrics['profit'] = metrics['total_amount'] - metrics['cost']
    metrics['profit_margin'] = (metrics['profit'] / metrics['total_amount'] * 100).round(2)
    return metrics

# Tool is now available for use in other compute nodes


In [ ]:
# ===== System-managed metadata (auto-generated, understand to edit) =====
# @node_type: tool
# @node_id: tool_promo_impact
# @execution_status: not_executed
# @name: Tool: Promotion Impact Analysis
# ===== End of system-managed metadata =====
import pandas as pd

def tool_promo_impact(orders_df):
    """Analyze the impact of promotions on sales"""
    promo_impact = orders_df.groupby('is_promoted').agg({
        'order_id': 'count',
        'total_amount': ['sum', 'mean'],
        'quantity': ['sum', 'mean'],
        'promotion_discount': 'sum'
    }).round(2)
    promo_impact.columns = ['order_count', 'total_sales', 'avg_order_value', 'total_qty', 'avg_qty', 'total_discount']
    return promo_impact

# Tool is now available for use in other compute nodes


In [ ]:
# ===== System-managed metadata (auto-generated, understand to edit) =====
# @node_type: compute
# @node_id: p1_daily_sales
# @execution_status: not_executed
# @depends_on: [load_orders_data]
# @name: Path 1: Daily Sales Trend
# ===== End of system-managed metadata =====
import pandas as pd

# Calculate daily sales
p1_daily_sales = load_orders_data.groupby('order_date').agg({
    'total_amount': 'sum',
    'order_id': 'count',
    'quantity': 'sum'
}).rename(columns={'order_id': 'order_count'}).reset_index()




In [ ]:
# ===== System-managed metadata (auto-generated, understand to edit) =====
# @node_type: compute
# @node_id: p1_category_sales
# @execution_status: not_executed
# @depends_on: [load_orders_data]
# @name: Path 1: Category Sales Distribution
# ===== End of system-managed metadata =====
import pandas as pd

# Sales by category
p1_category_sales = load_orders_data.groupby('category').agg({
    'total_amount': 'sum',
    'order_id': 'count',
    'quantity': 'sum'
}).rename(columns={'order_id': 'order_count'}).reset_index()

p1_category_sales = p1_category_sales.sort_values('total_amount', ascending=False)



In [ ]:
# ===== System-managed metadata (auto-generated, understand to edit) =====
# @node_type: compute
# @node_id: p1_weekly_trend
# @execution_status: not_executed
# @depends_on: [load_orders_data]
# @name: Path 1: Weekly Sales Trend
# ===== End of system-managed metadata =====
import pandas as pd

# Weekly aggregation
p1_weekly_trend = load_orders_data.copy()
p1_weekly_trend['week'] = p1_weekly_trend['order_date'].dt.to_period('W')
p1_weekly_trend = p1_weekly_trend.groupby('week').agg({
    'total_amount': 'sum',
    'order_id': 'count'
}).rename(columns={'order_id': 'order_count'}).reset_index()




In [ ]:
# ===== System-managed metadata (auto-generated, understand to edit) =====
# @node_type: chart
# @node_id: p1_sales_chart
# @execution_status: pending_validation
# @depends_on: [p1_category_sales]
# @name: Path 1: Sales by Category Chart
# ===== End of system-managed metadata =====
import plotly.express as px

# Create bar chart for category sales
p1_sales_chart = px.bar(
    p1_category_sales,
    x='category',
    y='total_amount',
    title='Sales Revenue by Category',
    labels={'total_amount': 'Total Sales ($)', 'category': 'Category'},
    color='order_count'
)

## 测试

In [ ]:
# ===== System-managed metadata (auto-generated, understand to edit) =====
# @node_type: compute
# @node_id: p2_customer_purchase
# @execution_status: validated
# @depends_on: [load_orders_data, load_customers_data]
# @name: Path 2: Customer Purchase Behavior
# ===== End of system-managed metadata =====import pandas as pd

# Merge orders and customers data
p2_customer_purchase = load_orders_data.merge(load_customers_data, on='customer_id')

# Customer purchase statistics
customer_stats = p2_customer_purchase.groupby('customer_id').agg({
    'total_amount': 'sum',
    'order_id': 'count',
    'quantity': 'sum',
    'customer_name': 'first',
    'loyalty_level': 'first'
}).rename(columns={'order_id': 'order_count'}).reset_index()

p2_customer_purchase = customer_stats.sort_values('total_amount', ascending=False)



In [ ]:
# ===== System-managed metadata (auto-generated, understand to edit) =====
# @node_type: compute
# @node_id: p2_loyalty_analysis
# @execution_status: validated
# @depends_on: [p2_customer_purchase]
# @name: Path 2: Loyalty Level Analysis
# ===== End of system-managed metadata =====import pandas as pd

# Analyze by loyalty level
p2_loyalty_analysis = p2_customer_purchase.groupby('loyalty_level').agg({
    'customer_id': 'count',
    'total_amount': ['sum', 'mean'],
    'order_count': 'mean',
    'quantity': 'sum'
}).round(2)

p2_loyalty_analysis.columns = ['customer_count', 'total_spending', 'avg_spending', 'avg_orders', 'total_quantity']
p2_loyalty_analysis = p2_loyalty_analysis.reset_index()



In [ ]:
# ===== System-managed metadata (auto-generated, understand to edit) =====
# @node_type: compute
# @node_id: p2_regional_behavior
# @execution_status: not_executed
# @depends_on: [p2_customer_purchase]
# @name: Path 2: Regional Purchase Behavior
# ===== End of system-managed metadata =====
import pandas as pd

# Regional analysis
p2_regional_behavior = p2_customer_purchase.groupby('region').agg({
    'customer_id': 'nunique',
    'total_amount': 'sum',
    'order_count': 'sum'
}).round(2).reset_index()

p2_regional_behavior.columns = ['region', 'unique_customers', 'total_sales', 'total_orders']


In [ ]:
# ===== System-managed metadata (auto-generated, understand to edit) =====
# @node_type: chart
# @node_id: p2_customer_chart
# @execution_status: not_executed
# @depends_on: [p2_loyalty_analysis]
# @name: Path 2: Loyalty Level Distribution Chart
# ===== End of system-managed metadata =====
import plotly.graph_objects as go

# Create pie chart for loyalty levels
p2_customer_chart = go.Figure(data=[go.Pie(
    labels=p2_loyalty_analysis['loyalty_level'],
    values=p2_loyalty_analysis['customer_count'],
    title='Customer Distribution by Loyalty Level'
)])



In [ ]:
# ===== System-managed metadata (auto-generated, understand to edit) =====
# @node_type: compute
# @node_id: p3_category_comparison
# @execution_status: not_executed
# @depends_on: [load_orders_data, load_products_data]
# @name: Path 3: Category Performance Comparison
# ===== End of system-managed metadata =====
import pandas as pd

# Merge and analyze category metrics
merged = load_orders_data.merge(load_products_data[['product_id', 'cost']], on='product_id')

p3_category_comparison = merged.groupby('category').agg({
    'total_amount': 'sum',
    'quantity': 'sum',
    'cost': 'sum',
    'order_id': 'count'
}).reset_index()

p3_category_comparison.columns = ['category', 'total_revenue', 'units_sold', 'total_cost', 'order_count']
p3_category_comparison['profit'] = p3_category_comparison['total_revenue'] - p3_category_comparison['total_cost']
p3_category_comparison['profit_margin'] = (p3_category_comparison['profit'] / p3_category_comparison['total_revenue'] * 100).round(2)
p3_category_comparison['avg_order_value'] = (p3_category_comparison['total_revenue'] / p3_category_comparison['order_count']).round(2)




In [ ]:
# ===== System-managed metadata (auto-generated, understand to edit) =====
# @node_type: compute
# @node_id: p3_category_market_share
# @execution_status: not_executed
# @depends_on: [p3_category_comparison]
# @name: Path 3: Category Market Share
# ===== End of system-managed metadata =====
import pandas as pd

# Calculate market share
total_revenue = p3_category_comparison['total_revenue'].sum()
p3_category_market_share = p3_category_comparison.copy()
p3_category_market_share['market_share_pct'] = (p3_category_market_share['total_revenue'] / total_revenue * 100).round(2)
p3_category_market_share = p3_category_market_share[['category', 'market_share_pct', 'profit_margin', 'avg_order_value']]




In [ ]:
# ===== System-managed metadata (auto-generated, understand to edit) =====
# @node_type: compute
# @node_id: p3_top_products
# @execution_status: not_executed
# @depends_on: [load_orders_data, load_products_data]
# @name: Path 3: Top Products by Revenue
# ===== End of system-managed metadata =====
import pandas as pd

# Top 10 products
merged = load_orders_data.merge(load_products_data[['product_id', 'product_name', 'category']], on='product_id')
p3_top_products = merged.groupby(['product_id', 'product_name', 'category']).agg({
    'total_amount': 'sum',
    'quantity': 'sum',
    'order_id': 'count'
}).rename(columns={'order_id': 'order_count'}).reset_index().sort_values('total_amount', ascending=False).head(10)




In [ ]:
# ===== System-managed metadata (auto-generated, understand to edit) =====
# @node_type: chart
# @node_id: p3_category_chart
# @execution_status: not_executed
# @depends_on: [p3_category_market_share]
# @name: Path 3: Market Share Distribution
# ===== End of system-managed metadata =====
import plotly.graph_objects as go

# Create donut chart for market share
p3_category_chart = go.Figure(data=[go.Pie(
    labels=p3_category_market_share['category'],
    values=p3_category_market_share['market_share_pct'],
    hole=.4,
    title='Market Share by Category (%)'
)])



In [ ]:
# ===== System-managed metadata (auto-generated, understand to edit) =====
# @node_type: compute
# @node_id: p4_promo_effectiveness
# @execution_status: not_executed
# @depends_on: [load_orders_data]
# @name: Path 4: Promotion Effectiveness
# ===== End of system-managed metadata =====
import pandas as pd

# Compare promoted vs non-promoted sales
p4_promo_effectiveness = load_orders_data.groupby('is_promoted').agg({
    'order_id': 'count',
    'total_amount': ['sum', 'mean'],
    'quantity': 'mean',
    'promotion_discount': 'sum'
}).round(2)

p4_promo_effectiveness.columns = ['order_count', 'total_sales', 'avg_order_value', 'avg_quantity', 'total_discount']
p4_promo_effectiveness = p4_promo_effectiveness.reset_index()
p4_promo_effectiveness['promo_status'] = p4_promo_effectiveness['is_promoted'].map({True: 'With Promotion', False: 'Without Promotion'})
p4_promo_effectiveness = p4_promo_effectiveness[['promo_status', 'order_count', 'total_sales', 'avg_order_value', 'total_discount']]




In [ ]:
# ===== System-managed metadata (auto-generated, understand to edit) =====
# @node_type: compute
# @node_id: p4_category_promo_impact
# @execution_status: not_executed
# @depends_on: [load_orders_data]
# @name: Path 4: Promotion Impact by Category
# ===== End of system-managed metadata =====
import pandas as pd

# Analyze promo impact by category
p4_category_promo_impact = load_orders_data.groupby(['category', 'is_promoted']).agg({
    'order_id': 'count',
    'total_amount': 'sum'
}).rename(columns={'order_id': 'order_count'}).reset_index()

p4_category_promo_impact = p4_category_promo_impact.pivot_table(
    index='category',
    columns='is_promoted',
    values=['order_count', 'total_amount'],
    fill_value=0
).reset_index()




In [ ]:
# ===== System-managed metadata (auto-generated, understand to edit) =====
# @node_type: compute
# @node_id: p4_payment_method_analysis
# @execution_status: not_executed
# @depends_on: [load_orders_data]
# @name: Path 4: Payment Method Analysis
# ===== End of system-managed metadata =====
import pandas as pd

# Payment method statistics
p4_payment_method_analysis = load_orders_data.groupby('payment_method').agg({
    'order_id': 'count',
    'total_amount': 'sum',
    'is_promoted': lambda x: (x == True).sum()
}).rename(columns={'order_id': 'order_count', 'is_promoted': 'promoted_orders'}).reset_index()

p4_payment_method_analysis = p4_payment_method_analysis.sort_values('total_amount', ascending=False)



In [ ]:
# ===== System-managed metadata (auto-generated, understand to edit) =====
# @node_type: chart
# @node_id: p4_promo_chart
# @execution_status: not_executed
# @depends_on: [p4_promo_effectiveness]
# @name: Path 4: Promotion Effectiveness Comparison
# ===== End of system-managed metadata =====
import plotly.graph_objects as go

# Create comparison chart
fig = go.Figure(data=[
    go.Bar(x=p4_promo_effectiveness['promo_status'], y=p4_promo_effectiveness['avg_order_value'],
           name='Average Order Value', yaxis='y1'),
    go.Bar(x=p4_promo_effectiveness['promo_status'], y=p4_promo_effectiveness['total_discount'],
           name='Total Discount', yaxis='y2')
])

fig.update_layout(
    title='Promotion Impact on Order Value',
    xaxis=dict(title='Promotion Status'),
    yaxis=dict(title='Average Order Value ($)'),
    yaxis2=dict(title='Total Discount ($)', overlaying='y', side='right'),
    hovermode='x unified'
)
p4_promo_chart = fig



In [ ]:
# @node_id: load_promotions_data
# @result_format: parquet
import pandas as pd
import os

# Load result from parquet
result_path = r'../projects/ecommerce_analytics/parquets/load_promotions_data.parquet'
if os.path.exists(result_path):
    load_promotions_data = pd.read_parquet(result_path)
    display(load_promotions_data)
else:
    print(f"Result file not found: {result_path}")

## ✓ Execution Complete: Load Promotions Data

**Completed at:** 2025-11-17T11:01:57.418029

**Execution time:** 1.19s

**Status:** ✅ Success

---

_Note: This documentation is auto-generated. For detailed AI-powered summaries, please enable summary generation in node settings._

In [ ]:
# @node_id: load_customers_data
# @result_format: parquet
import pandas as pd
import os

# Load result from parquet
result_path = r'../projects/ecommerce_analytics/parquets/load_customers_data.parquet'
if os.path.exists(result_path):
    load_customers_data = pd.read_parquet(result_path)
    display(load_customers_data)
else:
    print(f"Result file not found: {result_path}")

## ✓ Execution Complete: Load Customers Data

**Completed at:** 2025-11-17T11:02:06.730226

**Execution time:** 0.36s

**Status:** ✅ Success

---

_Note: This documentation is auto-generated. For detailed AI-powered summaries, please enable summary generation in node settings._

In [ ]:
# @node_id: load_orders_data
# @result_format: parquet
import pandas as pd
import os

# Load result from parquet
result_path = r'../projects/ecommerce_analytics/parquets/load_orders_data.parquet'
if os.path.exists(result_path):
    load_orders_data = pd.read_parquet(result_path)
    display(load_orders_data)
else:
    print(f"Result file not found: {result_path}")

## ✓ Execution Complete: Load Orders Data

**Completed at:** 2025-11-17T11:02:06.756248

**Execution time:** 0.03s

**Status:** ✅ Success

---

_Note: This documentation is auto-generated. For detailed AI-powered summaries, please enable summary generation in node settings._

In [ ]:
# @node_id: p2_customer_purchase
# @result_format: parquet
import pandas as pd
import os

# Load result from parquet
result_path = r'../projects/ecommerce_analytics/parquets/p2_customer_purchase.parquet'
if os.path.exists(result_path):
    p2_customer_purchase = pd.read_parquet(result_path)
    display(p2_customer_purchase)
else:
    print(f"Result file not found: {result_path}")

## ✓ Execution Complete: Path 2: Customer Purchase Behavior

**Completed at:** 2025-11-17T11:02:06.783578

**Execution time:** 0.42s

**Status:** ✅ Success

---

_Note: This documentation is auto-generated. For detailed AI-powered summaries, please enable summary generation in node settings._

In [ ]:
# @node_id: p2_loyalty_analysis
# @result_format: parquet
import pandas as pd
import os

# Load result from parquet
result_path = r'../projects/ecommerce_analytics/parquets/p2_loyalty_analysis.parquet'
if os.path.exists(result_path):
    p2_loyalty_analysis = pd.read_parquet(result_path)
    display(p2_loyalty_analysis)
else:
    print(f"Result file not found: {result_path}")

## ✓ Execution Complete: Path 2: Loyalty Level Analysis

**Completed at:** 2025-11-17T11:02:06.803949

**Execution time:** 0.91s

**Status:** ✅ Success

---

_Note: This documentation is auto-generated. For detailed AI-powered summaries, please enable summary generation in node settings._